In [ ]:
import os
os.environ["bilby_style"] = "none"

import bilby
import corner
import matplotlib.pyplot as plt
from nessai.livepoint import dataframe_to_live_points
from nessai.plot import corner_plot
import numpy as np

from nessai import config as nessai_config
nessai_config.plotting.disable_style = True

from thesis_utils.plotting import get_default_corner_kwargs, save_figure, set_plotting, get_default_figsize, get_corner_figsize
from thesis_utils.gw import get_cbc_parameter_labels

set_plotting()



In [ ]:
result_file = "outdir/fix-spins/paper_default_no_phase_marg_result.json"

In [ ]:
result = bilby.core.result.read_in_result(result_file)

In [ ]:
labels = ["phase", "psi", "theta_jn"]

In [ ]:
samples = dataframe_to_live_points(result.posterior[labels], non_sampling_parameters=False)

In [ ]:
corner_kwargs = get_default_corner_kwargs()
corner_kwargs["plot_datapoints"] = True
corner_kwargs["fill_contours"] = False
corner_kwargs["levels"] = None
corner_kwargs["show_titles"] = False
corner_kwargs["plot_datapoints"] = False

In [ ]:
fig = corner_plot(
    samples,
    labels=get_cbc_parameter_labels(samples.dtype.names),
    fig=plt.figure(figsize=get_corner_figsize(len(samples.dtype))),
    **corner_kwargs
)
save_figure(fig, "source_angles")

In [ ]:
from nessai.reparameterisations import CombinedReparameterisation
from nessai.gw.reparameterisations import get_gw_reparameterisation
from nessai.livepoint import empty_structured_array

In [ ]:
default_reparam_dict = {
    "phase": "angle-2pi",
    "psi": "angle-pi",
    "theta_jn": "angle-sine",
    "delta_phase": "angle-2pi",
}

In [ ]:
prior_bounds = {
    "phase": np.array([0, 2 * np.pi]),
    "delta_phase": np.array([0, 2 * np.pi]),
    "psi": np.array([0, np.pi]),
    "theta_jn": np.array([0, np.pi]),
}

## Defaults

In [ ]:
default_reparam = CombinedReparameterisation()
for parameter in labels:
    ReparamClass, kwargs = get_gw_reparameterisation(default_reparam_dict.get(parameter))
    reparam = ReparamClass(parameters=parameter, prior_bounds=prior_bounds, **kwargs)
    default_reparam.add_reparameterisation(reparam)

In [ ]:
x_prime_default = empty_structured_array(samples.size, names=default_reparam.prime_parameters, non_sampling_parameters=False)
log_j = np.zeros(samples.size)
_, x_prime_default, log_j = default_reparam.reparameterise(samples, x_prime_default, log_j)

In [ ]:
default_reparam.prime_parameters

In [ ]:
fig = corner_plot(
    x_prime_default,
    labels=[r"$x_{\phi_c}$", r"$y_{\phi_c}$", r"$x_{\psi}$", r"$y_{\psi}$", r"$\theta_{JN}'$"],
    fig=plt.figure(figsize=get_corner_figsize(len(x_prime_default.dtype))),
    **corner_kwargs
)
save_figure(fig, "nessai_angles_default")

## Alpha & beta

In [ ]:
samples_ab = empty_structured_array(samples.size, ["alpha", "beta", "theta_jn"], non_sampling_parameters=False)
samples_ab["alpha"] = (samples["psi"] + samples["phase"])#, 2 * np.pi)
samples_ab["beta"] = (samples["psi"] - samples["phase"])#, np.pi)
samples_ab["theta_jn"] = samples["theta_jn"]


In [ ]:
psi = (samples_ab["alpha"] + samples_ab["beta"]) / 2
phase = (samples_ab["alpha"] - samples_ab["beta"]) / 2

In [ ]:
fig = plt.figure()
plt.hist2d(psi, phase, 30)
plt.show()

In [ ]:
fig = corner_plot(
    samples_ab,
    labels=[r"$\alpha$", r"$\beta$", get_cbc_parameter_labels("theta_jn")],
    fig=plt.figure(figsize=get_corner_figsize(len(samples_ab.dtype))),
    **corner_kwargs
)
save_figure(fig, "alpha_beta")

## Delta phase

In [ ]:
samples_delta_phase = empty_structured_array(samples.size, ["delta_phase", "psi", "theta_jn"], non_sampling_parameters=False)
samples_delta_phase["psi"] = samples["psi"]
samples_delta_phase["theta_jn"] = samples["theta_jn"]
samples_delta_phase["delta_phase"] = samples["phase"] + np.sign(np.cos(samples["theta_jn"])) * samples["psi"]

In [ ]:
dp_reparam = CombinedReparameterisation()
for parameter in ["delta_phase", "psi", "theta_jn"]:
    ReparamClass, kwargs = get_gw_reparameterisation(default_reparam_dict.get(parameter))
    reparam = ReparamClass(parameters=parameter, prior_bounds=prior_bounds, **kwargs)
    dp_reparam.add_reparameterisation(reparam)

In [ ]:
dp_reparam

In [ ]:
delta_phase_cart = empty_structured_array(
    samples.size,
    dp_reparam.prime_parameters,
)
_, delta_phase_cart, _ = dp_reparam.reparameterise(samples_delta_phase, delta_phase_cart, 0.0)

In [ ]:
phase_re = np.mod(
    samples_delta_phase["delta_phase"] - np.sign(np.cos(samples["theta_jn"])) * samples["psi"],
    2 * np.pi,
)

In [ ]:
fig = corner_plot(
    samples_delta_phase,
    labels=get_cbc_parameter_labels(samples_delta_phase.dtype.names),
    fig=plt.figure(figsize=get_corner_figsize(len(samples_delta_phase.dtype))),
    **corner_kwargs
)
save_figure(fig, "delta_phase")

In [ ]:
quaternion_results = bilby.core.result.read_in_result("../quaternions/outdir/quaternions_2023_03_07_result.json")

In [ ]:
from thesis_utils.io import load_hdf5

In [ ]:
names = ["phase", "psi", "theta_jn", "delta_phase", "q_0", "q_1", "q_2", "q_3"]
samples_all = empty_structured_array(len(quaternion_results.posterior), names)
for name in names:
    if name in quaternion_results.posterior:
        samples_all[name] = quaternion_results.posterior[name]

In [ ]:
samples_all["delta_phase"] = samples_all["phase"] + np.sign(np.cos(samples_all["theta_jn"])) * samples_all["psi"]

In [ ]:
quat_nessai["posterior_samples"]

In [ ]:
corner_plot(samples_all, **corner_kwargs)